In [5]:
!pip uninstall telebot -y
!pip uninstall PyTelegramBotAPI -y
!pip install pyTelegramBotAPI
!pip install --upgrade pyTelegramBotAPI

Found existing installation: telebot 0.0.4
Uninstalling telebot-0.0.4:
  Successfully uninstalled telebot-0.0.4
Found existing installation: pyTelegramBotAPI 4.4.0
Uninstalling pyTelegramBotAPI-4.4.0:
  Successfully uninstalled pyTelegramBotAPI-4.4.0
  Using cached pyTelegramBotAPI-4.4.0-py3-none-any.whl


In [ ]:
import telebot
import bs4
import requests
import re

def get_data_from_csv():

    global average_pos
    global average_neg
    global top_pos
    global country
    global price_RUB
    global price_USD
    global all_data_list
    
    fh = open('cat.csv', 'r')
    initial_data = fh.readlines()

    data = []
    for item in initial_data[3:]:
        data.append(item.strip().split(','))

    price_data_raw = []
    price_data_raw.append(data.pop())
    price_data_raw.append(data.pop())

    country = data.pop()[1]

    data.pop()

    data_neg = []
    data_neg.append(data.pop())
    data_neg.append(data.pop())

    price_USD = [int(i) for i in price_data_raw[1][1].split('-')]
    price_RUB = [int(i) for i in price_data_raw[0][1].split('-')]

    pos_keys = [i[0] for i in data]
    pos_values = [int(i[1]) for i in data]
    average_pos = round(sum(pos_values) / len(pos_values), 1)

    data_pos = dict(zip(pos_keys, pos_values))
    top_pos_value = max(data_pos.values())
    top_pos = [x for x, y in data_pos.items() if y == top_pos_value]

    neg_keys = [i[0] for i in data_neg]
    neg_values = [int(i[1]) for i in data_neg]
    average_neg = round(sum(neg_values) / len(neg_values), 1)
    
    data_neg = dict(zip(neg_keys, neg_values))
    all_data = data_pos
    all_data.update(data_neg)
    
    all_data_list = []
    for k, v in all_data.items():
        all_data_list.append('\n' + k + ": " + str(v))
    all_data_list[0] = all_data_list[0].lstrip()

def parse_cats_index():
    url = 'https://petsi.net/cat-breeds'
    page = requests.get(url)

    from bs4 import BeautifulSoup
    soup = BeautifulSoup(page.text, 'lxml')

    global links
    links = []

    for link in soup.find_all('a'):
        if str(link.get('href')).startswith('/cat-breeds/'):
            links.append('https://petsi.net' + link.get('href'))

    links = links[::2]

    global cats
    cats = []

    for link in soup.find_all('a'):
        if link.get('data-title') != None:
            cat = link.get('data-title')
            latin = re.findall(r"([A-Za-z\s\-]+)", str(cat))
            entry = [x.strip() for x in latin if len(x) > 1]
            cats.append(entry)
    
    cats_single = []
    for i in cats:
        if len(i) == 1:
            cats_single.append(i[0])
        else:
            for j in i:
                cats_single.append(j)

cat_parsed = False

with open('cwt.txt') as fh:
    token = fh.read().strip()
bot = telebot.TeleBot(token)

@bot.message_handler(commands=['start'])
def show_start(message):
    try:
        parse_cats_index()
        bot.send_message(message.from_user.id, 'Привет! Я cat_woofer, и умею работать с сайтом, \
который знает все о кошках. Если вы введете название породы, я покажу ее самые яркие качества. \
И еще много интересного - увидите в подсказке')
        bot.send_message(message.from_user.id, 'Как правильно указать породу? Чтобы посмотреть доступные, \
Наберите /index и первую букву породы (например, /index s). Могу выдать все по команде /cats, но \
список длинный. Когда определитесь, жмите /go. \
Запустится штука под названием парсинг; надеюсь, все пройдет гладко! \
\nP.S. Список всех команд можно посмотреть, запустив /help')
    except:
        bot.send_message(message.from_user.id, 'Привет! Я cat_woofer, и я умею работать с сайтом, \
который знает все о кошках - но в этом случае он почему-то не сработал. Попробуйте перезапустить меня командой \
/start, если не будет получаться - что ж, придется отложить до лучших времен')
        
@bot.message_handler(commands=['help'])
def show_help(message):
    bot.send_message(message.from_user.id, '/index выдам список пород на определенную букву (добавьте ее через пробел, \
например index s)\n/cats выведу список (длинный!) всех-всех пород \n/go ввести породу и запустить парсинг; на выходе перечислю \
самые яркие черты \n/mean посчитаю среднюю качеств породы, положительных или отрицательных (пропишите + или - через \
пробел после команды) \n/all рейтинг всех качеств породы \
\n/from назову страну происхождения \n/price - если доступно, выдам цену в рублях или долларах \
(от вас rub или usd через пробел после команды)\
\n/pic пришлю картинку полюбоваться!\n/file запишу всю информацию в csv файл и, опять же, пришлю')

@bot.message_handler(commands=['index'])
def idx(message):
    try:
        parse_cats_index()
        msg_array = message.text.split()
        if len(msg_array) == 1:
            bot.send_message(message.from_user.id, 'Не понимаю, что искать... Через пробел после команды нужно ввести букву \
    на английском - например, \n/index s\nПопробуйте еще разок!')
        elif len(msg_array) > 2:
            bot.send_message(message.from_user.id, 'Как много всего, я запутался... Надо ввести только одну букву на английском \
    после команды - например, /index s \nПопробуйте еще разок!')
        else:
            letter = msg_array[1].lower()
            check_res_quantity = re.findall(r'\w', letter)
            if len(check_res_quantity) > 1:
                bot.send_message(message.from_user.id, 'Как много всего, я запутался... Надо ввести только одну букву на английском \
    после команды - например, /index s\nПопробуйте еще разок!')
            else:
                check_res_type = re.findall(r'[a-z]', letter)
                if len(check_res_type) == 0:
                    bot.send_message(message.from_user.id, 'Надо ввести букву на английском... Например, /index s\nПопробуйте \
    еще разок!')
                else:

                    cat_count = 0
                    cats_letter = []

                    for cat in cats:
                        for alias in cat:
                            if alias.lower().startswith(letter):
                                cat_count += 1
                                cats_letter.append(alias)

                    if cat_count == 0:
                        bot.send_message(message.from_user.id, 'К сожалению, я не знаю кошек на эту букву...')
                    else:
                        bot.send_message(message.from_user.id, f"Отлично! Вот какие кошки у меня есть на эту букву:")
                        bot.send_message(message.from_user.id, f"{', '.join(cats_letter)}")
                        bot.send_message(message.from_user.id, f"Для выбора породы жмите /go")
    except:
       bot.send_message(message.from_user.id, 'Что-то не так с моим главным ресурсом - веб-сайтом. Он не отвечает, \
поэтому, увы, помочь вам не смогу')

@bot.message_handler(commands=['cats'])
def show_cats(message):
    try:                              
        parse_cats_index()
        
        cats_single = []
        for i in cats:
            if len(i) == 1:
                cats_single.append(i[0])
            else:
                for j in i:
                    cats_single.append(j)
        
        bot.send_message(message.from_user.id, f"Внимание, длинный список всех знакомых мне пород:")
        bot.send_message(message.from_user.id, f"{', '.join(cats_single)}")
        bot.send_message(message.from_user.id, f"Для выбора породы жмите /go")
    except:
        bot.send_message(message.from_user.id, 'Что-то не так с моим главным ресурсом - веб-сайтом. Он не отвечает, \
поэтому, увы, помочь вам не смогу')

@bot.message_handler(commands=['go'])
def go(message):
    try:
        parse_cats_index()
        global cat_parsed
        cat_parsed = False
        bot.send_message(message.from_user.id, 'А теперь введите название породы (в любом регистре):')
    except:
        bot.send_message(message.from_user.id, 'Что-то не так с моим главным ресурсом - веб-сайтом. Он не отвечает, \
поэтому, увы, помочь вам не смогу') 
    
@bot.message_handler(commands=['file'])
def send_file(message):
    global cat_parsed
    if cat_parsed:
        fh = open('cat.csv', 'rb')
        bot.send_document(message.from_user.id, fh)
        fh.close()
    else:
        bot.send_message(message.from_user.id, 'Запрос на кошку не обработан - сделайте это при помощи команды \n/go')

@bot.message_handler(commands=['pic'])
def send_pic(message):
    global cat_parsed
    if cat_parsed:
        pic = open('cat_pic.jpg', 'rb')
        bot.send_photo(message.from_user.id, pic)
        fh.close()
    else:
        bot.send_message(message.from_user.id, 'Запрос на кошку не обработан - сделайте это при помощи команды \n/go')

@bot.message_handler(commands=['all'])
def send_all(message):
    global cat_parsed
    if cat_parsed:
        get_data_from_csv()
        bot.send_message(message.from_user.id, 'Так выглядит рейтинг всех качеств нашего кошака:')
        bot.send_message(message.from_user.id, f"{''.join(all_data_list)}")
    else:
        bot.send_message(message.from_user.id, 'Запрос на кошку не обработан - сделайте это при помощи команды \n/go')
    
@bot.message_handler(commands=['from'])
def show_country(message):
    global cat_parsed
    if cat_parsed:
        get_data_from_csv()
        bot.send_message(message.from_user.id, f'Страна происхождения красавицы - {country}')
    else:
        bot.send_message(message.from_user.id, 'Запрос на кошку не обработан - сделайте это при помощи команды \n/go')
        
@bot.message_handler(commands=['mean'])
def show_mean(message):
    global cat_parsed
    if cat_parsed:
        get_data_from_csv()
        msg = message.text.split()
        if len(msg) == 2:
            if msg[1] == '+':
                bot.send_message(message.from_user.id, f'Окей, средняя плюсов: {average_pos}')
            elif msg[1] == '-':
                bot.send_message(message.from_user.id, f'Так, средняя минусов: {average_neg}')
            else:
                bot.send_message(message.from_user.id, f'Хм, не пойму, чего же вы хотите... Все просто, \
командуйте /mean + или /mean -. А теперь попробуйте еще разок')
        else:
            bot.send_message(message.from_user.id, f'Вы не указали, хотите ли посчитать среднюю \
по положительным или отрицательным качествам - поэтому я буду считать, что вы любите кошек, и выдам среднюю плюсов: {average_pos}')
    else:
        bot.send_message(message.from_user.id, 'Запрос на кошку не обработан - сделайте это при помощи команды \n/go')

@bot.message_handler(commands=['price'])
def show_price(message):
    global cat_parsed
    if cat_parsed:
        if has_price:
            get_data_from_csv()
            msg = message.text.split()
            if len(msg) == 2:
                if msg[1].lower() == 'rub':
                    bot.send_message(message.from_user.id, f'В рублях: разброс от {price_RUB[0]} до {price_RUB[1]} т.р.')
                    bot.send_message(message.from_user.id, f'Перед покупкой загляните, пожалуйста, в ближайший кошачий приют - \
быть может ваш друг ждет вас там!')
                elif msg[1].lower() == 'usd':
                    bot.send_message(message.from_user.id, f'В USD: разброс от {price_USD[0]} до {price_USD[1]}')
                    bot.send_message(message.from_user.id, f'Перед покупкой загляните, пожалуйста, в ближайший кошачий приют - \
быть может ваш друг ждет вас там!')
                else:
                    bot.send_message(message.from_user.id, f'Хм, не пойму, чего же вы хотите... Все просто, \
командуйте /price rub или /price usd. А теперь попробуйте еще разок')
            else:
                bot.send_message(message.from_user.id, f'Вы не указали, хотите ли цену в рублях или долларах \
- поэтому я буду считать, что в России нам все-таки привычнее в рублях: разброс от {price_RUB[0]} до {price_RUB[1]} т.р.')
                bot.send_message(message.from_user.id, f'Перед покупкой загляните, пожалуйста, в ближайший кошачий приют - \
быть может ваш друг ждет вас там!')
        else: 
            bot.send_message(message.from_user.id, 'К сожалению, цена для данной кошки неизвестна \
Но это не страшно - все равно я хотел попросить вас перед покупкой заглянуть в ближайший кошачий приют. \
Очень может быть, что ваш настоящий друг ждет вас именно там!')
    else:
        bot.send_message(message.from_user.id, 'Запрос на кошку не обработан - сделайте это при помощи команды \n/go')

@bot.message_handler(content_types=['text'])
def get_text_message(message):
    global cat_parsed
    cat_request = message.text 
    search_flag = False
    
    if not cat_parsed:

        for i in range(len(cats)):
            for j in range(len(cats[i])):
                if cats[i][j].lower() == cat_request.lower():
                    url_cat = links[i]
                    search_flag = True

        if search_flag:
            try:
                #парсинг страницы отдельной кошки
                page_cat = requests.get(url_cat)

                from bs4 import BeautifulSoup
                soup_cat = BeautifulSoup(page_cat.text, 'lxml')

                traits = []

                for label in soup_cat.find_all('div', class_='param-dot-pane-item__label'):
                    traits.append(label.text)

                grades = []

                for tag in soup_cat.find_all('span', class_='clinic-card__stars br-wrapper br-theme-fontawesome-stars'):
                    grades.append(int(tag.find('span')['class'][2].split('-')[-1]))

                traits_neg = []
                traits_neg.append(traits.pop(5))
                traits_neg.append(traits.pop())
                grades_neg = []
                grades_neg.append(grades.pop(5))
                grades_neg.append(grades.pop())

                cat_dict_pos = dict(zip(traits, grades))
                cat_dict_neg = dict(zip(traits_neg, grades_neg))

                #парсинг картинки
                pic_url = 'https://petsi.net' + str(soup_cat.find_all('img', {'itemprop': 'thumbnailUrl'})[0]['src'])

                pic = requests.get(pic_url)
                with open('cat_pic.jpg', 'wb') as pic_file:
                    pic_file.write(pic.content)

                #парсинг страны происхождения
                country_origin = soup_cat.find_all('table', {'class': 'breed-view__table-info'})[0].find_all('td')[1].text

                #парсинг цен
                prices_raw = soup_cat.find_all('table', {'class': 'breed-view__table-info'})[0].find_all('td')[-1].text    
                prices = [int(x) for x in re.findall(r'\d+', prices_raw)]
                if len(prices) > 1:
                     global has_price
                     has_price = True

                #парсинг сайта ЦБ для пересчета долларовой цены в рубли
                url_rate = 'https://cbr.ru'
                page_rate = requests.get(url_rate)

                from bs4 import BeautifulSoup
                soup_rate = BeautifulSoup(page_rate.text, 'lxml')

                rates = soup_rate.find_all('div', {'data-tooltip-content' : 'Официальный курс Банка России'})[0]
                rate_raw = rates.find_all('div', {'class': 'indicator_el_value mono-num'})[-1].text
                rate = float('.'.join(re.findall(r'\d+', rate_raw)))

                prices_rub = [int(round(x * rate / 1000, 0))  for x in prices]

                with open('cat.csv', 'w', encoding='utf8') as file:
                    file.write(f'Данные о нашей кошке, {cat_request}\n')
                    file.write(' \n')
                    file.write('Качество, Оценка\n')
                    for key, value in cat_dict_pos.items():
                        file.write('{0},{1}\n'.format(key, value))
                    for key, value in cat_dict_neg.items():
                        file.write('{0},{1}\n'.format(key, value))
                    file.write(' \n')
                    file.write(f'Страна происхождения:, {country_origin}\n')
                    if has_price:
                        file.write(f'Диапазон цен в USD:, {prices[0]} - {prices[1]}\n')              
                        file.write(f'Диапазон цен в RUB (тыс.):, {prices_rub[0]} - {prices_rub[1]}\n')
                    
                cat_parsed = True
                bot.send_message(message.from_user.id, 'Все отлично, данные у меня. Итак, самые яркие качества породы:')
                get_data_from_csv()
                bot.send_message(message.from_user.id, f"{', '.join(top_pos)}")
                bot.send_message(message.from_user.id, 'А теперь выберите, пожалуйста, что хотите дальше:')
                if has_price:
                    bot.send_message(message.from_user.id, '\n/all выдам рейтинг всех качеств \
\n/mean посчитаю среднюю положительных или отрицательных (добавьте + или - через пробел после команды) \
\n/from назову страну происхождения \n/price - если доступно, выдам цену в рублях или долларах \
(от вас rub или usd через пробел после команды)\
\n/pic пришлю картинку полюбоваться!\n/file запишу всю информацию в csv файл и, опять же, пришлю \
\nДля выбора новой породы - /go')
                else:
                    bot.send_message(message.from_user.id, '\n/all выдам рейтинг всех качеств \
\n/mean посчитаю среднюю положительных или отрицательных (добавьте + или - через пробел после команды) \
\n/from назову страну происхождения \
\n/pic пришлю картинку полюбоваться!\n/file запишу всю информацию в csv файл и, опять же, пришлю \
\nДля выбора новой породы - /go')
                     
            except:
                cat_parsed = False
                bot.send_message(message.from_user.id, 'Сайт сплоховал - к сожалению, у меня не получилось собрать данные \
Попробуйте еще раз, если снова нет - возвращайтесь через какое-то время')    

        else:
            bot.send_message(message.from_user.id, 'Хм, такой кошки я не знаю... Давайте-ка поробуем еще раз. Жмите /go')
    else:
        bot.send_message(message.from_user.id, 'Не понимаю команду... Давайте еще раз напомню, что могу сделать:') 
        if has_price:
            bot.send_message(message.from_user.id, '\n/top перечислю самые выдающиеся положительные качества  \
\n/from назову страну происхождения породы\n/buy - если доступно, выдам цену в долларах или рублях \
(от вас usd  или rub через пробел после команды)\
\n/pic пришлю картинку полюбоваться!\n/file запишу всю информацию в csv файл и, опять же, пришлю')
        else:
            bot.send_message(message.from_user.id, '\n/top перечислю самые выдающиеся положительные качества  \
\n/from назову страну происхождения породы \
\n/pic пришлю картинку полюбоваться!\n/file запишу всю информацию в csv файл и, опять же, пришлю')

bot.polling(none_stop=True, interval=0)